In [1]:
from OpenMiChroM.ChromDynamics import MiChroM

In [2]:
sim = MiChroM(name='stomach_GRCh38', temperature=1.0, timeStep=0.01)

    ***************************************************************************************     
     **** **** *** *** *** *** *** *** OpenMiChroM-1.1.01 *** *** *** *** *** *** **** ****     

         OpenMiChroM is a Python library for performing chromatin dynamics simulations.         
                            OpenMiChroM uses the OpenMM Python API,                             
                employing the MiChroM (Minimal Chromatin Model) energy function.                
      The chromatin dynamics simulations generate an ensemble of 3D chromosomal structures      
      that are consistent with experimental Hi-C maps, also allows simulations of a single      
                 or multiple chromosome chain using High-Performance Computing                  
                            in different platforms (GPUs and CPUs).                             

         OpenMiChroM documentation is available at https://open-michrom.readthedocs.io          

         OpenMiChroM is des

In [3]:
sim.setup(platform="Cuda")

Using platform: CUDA


In [4]:
sim.saveFolder('/home/antonio/test_OM1.1')

In [5]:
sim.buildClassicMichrom(ChromSeq='inputs/stomach_GRCh38.bed', chromosome='chr11')


FENEBond was added
AngleForce was added
RepulsiveSoftCore was added
TypetoType was added
IdealChromosome was added
FlatBottomHarmonic was added
Setting positions... loaded!
Setting velocities... loaded!
Context created!

Simulation name: stomach_GRCh38
Number of beads: 2702, Number of chains: 1
Potential energy: 63.93571, Kinetic Energy: 1.51567 at temperature: 1.0

Potential energy per forceGroup:
                                  Values
FENEBond                   55114.613361
AngleForce                     0.993050
RepulsiveSoftCore              0.000000
TypetoType                  -216.843586
IdealChromosome               -1.082162
FlatBottomHarmonic        117856.612623
Potential Energy (total)  172754.293286


In [6]:
sim.createReporters(statistics=True, traj=False, trajFormat="swb", energyComponents=True, interval=1000)


In [7]:
sim.run(nsteps=10**5, report=True, interval=10**4) #inbetween

#"Progress (%)"	"Step"	"Speed (ns/day)"	"Time Remaining"
10.0%	10000	0	--
20.0%	20000	1.46e+04	0:04
30.0%	30000	1.44e+04	0:04
40.0%	40000	1.38e+04	0:03
50.0%	50000	1.34e+04	0:03
60.0%	60000	1.3e+04	0:02
70.0%	70000	1.28e+04	0:02
80.0%	80000	1.26e+04	0:01
90.0%	90000	1.25e+04	0:00
100.0%	100000	1.24e+04	0:00
